In [1]:
# https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8
import pandas as pd
import json
#pd.set_option('display.max_columns', 30)

import glob, os, json


json_dir = '/home/fedorovn19/github/Y_Swap/algorand_solutions/data'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

In [104]:
dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
    dfs.append(json_data)

df = pd.concat(dfs)

In [105]:
df.describe()

,close-rewards,closing-amount,confirmed-round,fee,first-valid,intra-round-offset,last-valid,receiver-rewards,round-time,sender-rewards,asset-transfer-transaction.amount,asset-transfer-transaction.asset-id,asset-transfer-transaction.close-amount,signature.logicsig.multisig-signature.threshold,signature.logicsig.multisig-signature.version,signature.multisig.threshold,signature.multisig.version
count,66855.0,66855.0,6.685500e+04,66855.000000,6.685500e+04,66855.000000,6.685500e+04,66855.0,6.685500e+04,6.685500e+04,6.685500e+04,66855.0,66855.0,8922.0,8922.0,72.000000,72.0
mean,0.0,0.0,1.448745e+07,1128.017186,1.448745e+07,25.285020,1.448845e+07,0.0,1.623993e+09,3.112819e+06,1.119281e+11,31566704.0,0.0,2.0,1.0,2.527778,1.0
std,0.0,0.0,1.721471e+06,5559.441123,1.721471e+06,32.240918,1.721471e+06,0.0,7.567248e+06,2.645707e+08,9.036957e+11,0.0,0.0,0.0,0.0,0.502731,0.0
min,0.0,0.0,1.161112e+07,1000.000000,1.161112e+07,0.000000,1.161212e+07,0.0,1.611337e+09,0.000000e+00,2.000000e+00,31566704.0,0.0,2.0,1.0,2.000000,1.0
25%,0.0,0.0,1.296338e+07,1000.000000,1.296338e+07,10.000000,1.296438e+07,0.0,1.617285e+09,0.000000e+00,1.648000e+07,31566704.0,0.0,2.0,1.0,2.000000,1.0
50%,0.0,0.0,1.448360e+07,1000.000000,1.448360e+07,20.000000,1.448460e+07,0.0,1.623989e+09,0.000000e+00,3.716100e+08,31566704.0,0.0,2.0,1.0,3.000000,1.0
75%,0.0,0.0,1.624680e+07,1000.000000,1.624680e+07,34.000000,1.624780e+07,0.0,1.631729e+09,3.036000e+03,7.249085e+09,31566704.0,0.0,2.0,1.0,3.000000,1.0
max,0.0,0.0,1.671100e+07,261000.000000,1.671099e+07,2353.000000,1.671199e+07,0.0,1.633758e+09,3.691126e+10,9.700000e+13,31566704.0,0.0,2.0,1.0,3.000000,1.0


In [98]:
df.head(5)

,close-rewards,closing-amount,confirmed-round,fee,first-valid,genesis-hash,id,intra-round-offset,last-valid,lease,receiver-rewards,round-time,sender,sender-rewards,tx-type,asset-transfer-transaction.amount,asset-transfer-transaction.asset-id,asset-transfer-transaction.close-amount,asset-transfer-transaction.receiver,signature.sig,genesis-id,signature.logicsig.args,signature.logicsig.logic,signature.logicsig.multisig-signature.subsignature,signature.logicsig.multisig-signature.threshold,signature.logicsig.multisig-signature.version,group,note,asset-transfer-transaction.close-to
0,0,0,12218837,1000,12218833,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,VLSBKGSBIH6L26WZOBRI6GW2Q5DYOSY3LXULUJZQZJDO4U...,5,12219833,FmwXyV8lDnFVcf7T605HZ9E+KUeWwHM3D685rBYqbKw=,0,1614012493,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3...,1020,axfer,27220210000,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQM...,NLJ5g+XZBv4YC4A1kxzsAVMhOwpAhyMb9fQTzNc1ZxgEAf...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,12218850,1000,12218847,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,F5S5OXRJBT3J2YBBG4C3SNH36HO6THDABNZ4Q2OTHZYH56...,28,12219847,oPeJWdsPkkRd+zJSuvWZS3QRPJEsAkjhfggaV778NqA=,0,1614012550,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3...,0,axfer,164960000,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQM...,OzPuXOInYROdh0cg+tLEIBl4VZ0n8MRsiRXAg2VDD3DBaR...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,12218891,1000,12218888,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,MNLZJL6TEJP3P4EFIAGU73U2DAVONKBJJZ2WDSITTVXO5P...,75,12219888,wYHxfxq09fuIiFXQzH+F+AotLwioo010nPJAyIaxdfI=,0,1614012730,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3...,0,axfer,29390000,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQM...,BdQmcuANhGTOvWL91G9KBFJ8jlirt+0eMmHUfQ1voyoR3t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,12218933,1000,12218929,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,D4OR7MOQJO56JSMSPHPZQNMDCL4QB4DBX2NEX2NPCMKTCZ...,10,12219929,GbCFK5sPnlcyZ1pYzo7Cunxg8jqgiPI0ZS1fkXpj5e8=,0,1614012913,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3...,1020,axfer,12310000,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQM...,RdNAR4vdN6lotFpyroAt2jhzbT5caHB2NwaEXBlmKxgEJN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,12218946,1000,12218943,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,7NCPUO5XSNM4STRM7HGHNZGXPQ6P4AFIQUCLCO32WWAVOW...,72,12219943,FC6NUgMh7U3P5YvPFZPvGViNvlcNdSGP34u0qTfQ/f0=,0,1614012971,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3...,0,axfer,184080000,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQM...,mv+ZWb10V1C1vcb+vBsHLwP2vEXc/mnshikJGwqabM2tvG...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
df.describe()

,close-rewards,closing-amount,confirmed-round,fee,first-valid,intra-round-offset,last-valid,receiver-rewards,round-time,sender-rewards,asset-transfer-transaction.amount,asset-transfer-transaction.asset-id,asset-transfer-transaction.close-amount,signature.logicsig.multisig-signature.threshold,signature.logicsig.multisig-signature.version
count,14049.0,14049.0,1.404900e+04,14049.000000,1.404900e+04,14049.000000,1.404900e+04,14049.0,1.404900e+04,1.404900e+04,1.404900e+04,14049.0,14049.0,3068.0,3068.0
mean,0.0,0.0,1.208854e+07,1052.212969,1.208854e+07,21.073101,1.208954e+07,0.0,1.613439e+09,6.348657e+04,8.478168e+10,31566704.0,0.0,2.0,1.0
std,0.0,0.0,2.383798e+05,3093.449469,2.383797e+05,18.608023,2.383798e+05,0.0,1.047593e+06,2.642185e+06,7.669663e+11,0.0,0.0,0.0,0.0
min,0.0,0.0,1.161112e+07,1000.000000,1.161112e+07,0.000000,1.161212e+07,0.0,1.611337e+09,0.000000e+00,9.000000e+00,31566704.0,0.0,2.0,1.0
25%,0.0,0.0,1.192527e+07,1000.000000,1.192527e+07,8.000000,1.192627e+07,0.0,1.612722e+09,0.000000e+00,1.360000e+07,31566704.0,0.0,2.0,1.0
50%,0.0,0.0,1.208449e+07,1000.000000,1.208449e+07,17.000000,1.208549e+07,0.0,1.613422e+09,1.018000e+03,3.062200e+08,31566704.0,0.0,2.0,1.0
75%,0.0,0.0,1.223984e+07,1000.000000,1.223984e+07,29.000000,1.224084e+07,0.0,1.614105e+09,2.042000e+03,6.872840e+09,31566704.0,0.0,2.0,1.0
max,0.0,0.0,1.260977e+07,261000.000000,1.260977e+07,383.000000,1.261077e+07,0.0,1.615729e+09,2.400520e+08,4.698550e+13,31566704.0,0.0,2.0,1.0


In [101]:
# Number of Unique Rounds in the data
df['confirmed-round'].nunique()

13755

In [43]:
df['round-time'] = pd.to_datetime(df['round-time'], unit='s')

In [71]:
ag_df = df.groupby(by=[df['round-time'].dt.date])['asset-transfer-transaction.amount'].agg(volume='sum', mean='mean')
ag_df

,volume,mean
round-time,,
2021-01-22,19274045960000,2.753435e+11
2021-01-23,20364109550000,4.525358e+11
2021-01-24,1661494340000,7.552247e+10
2021-01-25,13488398930000,8.536961e+10
2021-01-26,9178004730000,3.121770e+10
2021-01-27,9007836700000,3.833122e+10
2021-01-28,20047076000000,8.567126e+10
2021-01-29,45832745850000,1.235384e+11
2021-01-30,1049019530000,7.828504e+09
